# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [173]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [174]:
MovieLens_df = pd.read_table("ratings.dat", sep='::', header=None, names=['UserID','MovieID','Rating','Timestamp'])
MovieLens_df.drop(['Timestamp'], axis=1, inplace=True)
MovieLens_pivot = MovieLens_df.pivot_table(index='UserID', columns='MovieID', values='Rating')
row_index = MovieLens_pivot.isnull().sum(axis = 1).sort_values() # NaN이 적은 순으로 행 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, index=row_index.index)
# rating이 많은 사용자 1,000명
MovieLens_pivot = MovieLens_pivot.iloc[:1000,:]
col_index = MovieLens_pivot.isnull().sum(axis = 0).sort_values() # NaN이 적은 순으로 열 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, columns=col_index.index)
# rating이 많은 영화 1,000개
MovieLens_pivot = MovieLens_pivot.iloc[:,:1000]
# user_index = MovieLens_pivot.index.values # User ID 순서 기억하기
# user_index = pd.Series(user_index) # Index rename Simi인자로 넣기 위해 형태 변경
MovieLens_pivot.fillna(0, inplace=True)
MovieLens_pivot = MovieLens_pivot.astype('int64')
MovieLens_pivot = np.array(MovieLens_pivot) # 함수 인자로 넣기 위해 numpy array 형태로 변경

# 1. Cosine Similarity

In [202]:
def COS(data):
    num_users = np.size(data, axis=0)
    simCOS = np.zeros((num_users,num_users))  # 0으로 초기화 된 행렬 생성
    for u in range(0, num_users):  # 각 유저별로 for문 반복
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis=None))

            U = np.delete(data[u, ], arridx)
            V = np.delete(data[v, ], arridx)

            if np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0:
                simCOS[u, v] = 0
            else:
                simCOS[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))

            simCOS[v, u] = simCOS[u, v]
    return simCOS

In [203]:
COS(A)

array([[0.        , 0.9165204 , 0.79442991, 0.63943019, 0.88069603],
       [0.9165204 , 0.        , 0.80671783, 0.82956136, 0.89472029],
       [0.79442991, 0.80671783, 0.        , 0.71842121, 0.84181887],
       [0.63943019, 0.82956136, 0.71842121, 0.        , 0.79530788],
       [0.88069603, 0.89472029, 0.84181887, 0.79530788, 0.        ]])

In [224]:
def PCC(data):
    num_users = np.size(data, axis=0)
    simPCC = np.zeros((num_users,num_users))  # 0으로 초기화 된 행렬 생성
    mean = np.nanmean(np.where(data != 0, data, np.nan), axis=1)
    for u in range(0, num_users):
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis=None))

            U = np.delete(data[u, ], arridx) - mean[u]
            V = np.delete(data[v, ], arridx) - mean[v]

            if np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0:
                simPCC[u, v] = 0
            else:
                simPCC[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))

            simPCC[v, u] = simPCC[u, v]

    return simPCC

# 2. Pearson Correlation Coefficient

In [128]:
PCC(MovieLens_pivot)

array([[ 0.        ,  0.65707268,  0.26630576, ...,  0.02698069,
         0.11686485,  0.094992  ],
       [ 0.65707268,  0.        ,  0.25738696, ...,  0.02741278,
         0.15325623, -0.09991021],
       [ 0.26630576,  0.25738696,  0.        , ...,  0.13864556,
         0.16625119,  0.15649389],
       ...,
       [ 0.02698069,  0.02741278,  0.13864556, ...,  0.        ,
        -0.14768909,  0.29004726],
       [ 0.11686485,  0.15325623,  0.16625119, ..., -0.14768909,
         0.        , -0.35809302],
       [ 0.094992  , -0.09991021,  0.15649389, ...,  0.29004726,
        -0.35809302,  0.        ]])

# Rating

In [209]:
A = np.array([[4, 1, 1, 4, 2, 3, 5, 0, 4, 0],
              [0, 4, 2, 0, 3, 2, 5, 0, 4, 3],
              [2, 0, 4, 5, 2, 0, 1, 3, 4, 2],
              [1, 4, 0, 1, 4, 5, 0, 3, 1, 2],
              [1, 2, 3, 4, 0, 3, 4, 4, 2, 5]])
A

array([[4, 1, 1, 4, 2, 3, 5, 0, 4, 0],
       [0, 4, 2, 0, 3, 2, 5, 0, 4, 3],
       [2, 0, 4, 5, 2, 0, 1, 3, 4, 2],
       [1, 4, 0, 1, 4, 5, 0, 3, 1, 2],
       [1, 2, 3, 4, 0, 3, 4, 4, 2, 5]])

In [208]:
COS(A)

array([[0.        , 0.9165204 , 0.79442991, 0.63943019, 0.88069603],
       [0.9165204 , 0.        , 0.80671783, 0.82956136, 0.89472029],
       [0.79442991, 0.80671783, 0.        , 0.71842121, 0.84181887],
       [0.63943019, 0.82956136, 0.71842121, 0.        , 0.79530788],
       [0.88069603, 0.89472029, 0.84181887, 0.79530788, 0.        ]])

In [302]:
# 임시로 만들어 놓기
def basic_mean(mat, sim, k):
    predicted_rating = np.zeros(mat.shape)
    
    mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis=1) # mean of user

    if(sim == 'COS'):
        Sim = COS(mat)
    elif(sim == 'PCC'):
        Sim = PCC(mat)
        
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)

    NumUsers = np.size(mat, axis=0)
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u]]
        list_rating = mat[k_neighbors[u]]
        list_mean = mean[k_neighbors[u]]
        
        mom = np.sum(list_sim)
        sun = np.sum(list_sim.reshape(-1,1) * (list_rating - list_mean.reshape(-1,1)), axis=0)
        predicted_rating[u] = mean[u] + sun / mom
        
    return predicted_rating

In [235]:
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9
0,4,1,1,4,2,3,5,0,4,0
1,0,4,2,0,3,2,5,0,4,3
2,2,0,4,5,2,0,1,3,4,2
3,1,4,0,1,4,5,0,3,1,2
4,1,2,3,4,0,3,4,4,2,5


In [243]:
pd.DataFrame(basic_mean(A, 'COS', 3))

,0,1,2,3,4,5,6,7,8,9
0,0.852409,1.993736,2.852409,2.791476,1.573520,1.626269,3.333557,2.178407,3.219876,3.272625
1,2.407053,2.647072,1.729349,3.423472,2.316521,3.994133,3.456401,2.663491,2.745859,2.688165
2,1.387715,2.077614,1.761762,2.421481,1.383407,2.412143,4.397775,1.120719,3.053186,2.455969
3,0.477036,1.619323,2.477036,2.414989,1.199685,1.250690,2.958721,1.801816,2.845669,2.896674
4,2.042926,1.757570,2.360433,3.007859,2.395500,1.746853,3.767066,1.018576,4.053643,1.722503


In [247]:
tmp = pd.DataFrame(MovieLens_pivot)
tmp.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,5,5,4,4,5,4,4,5,5,5,...,2,4,2,3,3,3,3,0,0,4
1,5,2,3,4,3,3,5,5,5,5,...,4,3,4,5,2,4,3,1,0,5
2,5,5,5,4,4,5,5,5,5,5,...,5,3,4,4,0,0,3,4,0,5
3,5,5,5,5,5,3,3,5,1,4,...,3,1,0,4,0,2,4,3,3,0
4,4,4,3,4,3,5,5,2,3,3,...,2,2,3,2,0,0,0,2,0,3


In [246]:
pd.DataFrame(basic_mean(MovieLens_pivot, 'COS', 10))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.087396,3.786176,2.387602,1.487075,2.385845,1.787562,1.087888,1.187664,3.889803,3.989177,...,-0.210449,-0.510589,-0.510589,-0.510589,-0.510589,-0.211301,-0.510589,-0.011537,-0.110405,0.789633
1,2.187077,2.786341,1.986564,2.385816,1.986982,1.384910,1.086462,1.886194,4.285793,3.983600,...,-0.212213,-0.513783,-0.513783,-0.513783,-0.513783,-0.513783,-0.513783,0.284964,0.387002,0.385253
2,4.823643,4.424384,4.123968,3.222763,3.723005,3.924064,3.222502,3.622439,3.824250,4.023813,...,0.524163,0.824155,0.123672,0.823142,0.123672,0.723272,0.423289,0.523162,0.523698,0.924581
3,3.362624,3.061314,3.262264,1.463868,2.162297,1.964213,1.363241,2.263711,1.763739,3.164892,...,-0.436822,-0.436619,-0.836057,-0.836057,-0.434887,-0.836057,-0.135531,0.464061,-0.337013,-0.436582
4,1.924100,3.225046,1.323420,1.324805,1.324540,1.925261,0.824861,1.125092,2.825312,3.025055,...,-0.876078,-0.776677,-0.876472,-1.175858,-1.175858,-1.175858,-1.175858,-0.875718,-0.875555,-0.375222
5,2.795212,2.195189,2.294617,1.195728,1.894776,0.896186,1.895391,1.195865,2.498401,2.597237,...,-0.803290,-0.803904,-1.103440,-0.104408,-1.103440,-0.503846,-1.103440,-0.204765,-1.103440,-0.604408
6,3.623221,3.122418,3.223495,2.922103,3.022329,2.522371,2.722387,3.122130,2.723431,1.923684,...,-0.177321,0.422422,-0.576560,1.123593,-0.576560,0.023779,-0.176740,-0.276889,-0.576560,-0.575809
7,3.314362,3.513850,2.914775,2.415628,3.214722,3.513850,2.815917,2.113789,3.021991,3.417410,...,-0.582265,-0.980829,-0.082721,-0.980829,-0.980829,-0.679557,-0.980829,-0.980829,-0.581854,-0.980829
8,0.770112,2.169865,0.870651,-0.324715,1.169307,0.769145,0.571348,-0.626370,3.174325,1.877944,...,-1.226279,-1.225448,-1.525743,-1.126613,-1.525743,-1.525743,-1.326001,-1.525743,-1.525743,-1.127478
9,2.523047,3.322284,2.822359,1.920694,1.623494,0.824525,1.521331,1.122157,3.924644,4.023566,...,-0.175610,-0.475919,-0.475919,-0.475919,-0.475919,-0.475919,-0.475919,0.022872,0.323057,0.325734


In [248]:
pd.DataFrame(basic_mean(MovieLens_pivot.T, 'COS', 10))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.339336,3.135496,4.635508,2.641939,3.136678,1.937022,2.338958,2.139918,3.138258,2.739921,...,3.640967,1.843438,0.437322,2.241916,2.341858,2.341535,1.438221,0.533456,1.340918,3.438753
1,3.523317,3.223526,4.321664,2.924619,3.620137,2.520902,2.023438,1.425439,3.320781,2.523918,...,3.025386,1.626592,0.620824,2.425094,2.823962,2.126022,1.323384,0.623268,1.020368,3.521498
2,3.898945,2.597887,4.697854,3.797163,2.797054,2.895825,2.997439,3.198952,2.197166,3.898414,...,3.299092,2.998163,0.197327,2.898331,2.298527,2.898016,1.896899,0.698583,0.697495,3.197118
3,3.612872,3.112963,3.912647,3.612255,2.412450,1.812786,3.012809,2.211738,1.612439,4.112579,...,1.812244,1.812601,0.511705,1.213110,1.212034,2.813278,1.112642,0.911758,0.512847,2.212257
4,3.583691,2.284768,4.480335,3.284879,2.981663,2.579810,2.782510,2.085802,2.383668,2.486832,...,2.985583,2.089049,0.178549,2.388259,2.184352,3.085719,1.483941,1.080047,0.682717,2.681369
5,3.496959,2.400169,4.697890,3.197939,3.197654,1.896424,2.498007,2.398066,2.898931,2.199014,...,3.499985,1.798729,0.298471,1.598967,2.400974,2.798191,1.298344,1.198175,0.397960,2.999495
6,3.734086,2.637492,4.834128,4.234624,3.136602,2.833323,2.733968,2.534796,2.034142,3.834762,...,2.734996,2.933377,0.232787,2.132545,2.035204,3.735613,1.435030,0.632153,0.333803,2.434158
7,3.943451,3.543054,4.443315,4.143806,3.442457,1.741205,2.743366,1.943480,1.443199,3.243198,...,2.142409,3.043975,0.342588,1.445069,1.542937,3.443590,1.441440,0.743178,0.243031,1.343242
8,3.831788,3.033350,4.832614,2.231498,2.531942,2.630922,2.331431,2.233584,3.133298,3.431947,...,2.234978,1.231768,0.432319,1.231876,2.933976,1.432484,1.333211,3.033729,0.032892,3.732310
9,4.258346,3.160653,4.459412,2.557861,3.656791,3.358707,2.958246,1.557327,3.659811,2.959735,...,0.559688,0.458288,0.058896,0.558187,2.160538,2.256732,0.857826,1.461344,0.558221,3.959869


In [249]:
pd.DataFrame(basic_baseline(MovieLens_pivot.T, 'COS', 10))

[[  1  11   4 ... 833 834   0]
 [  0  11   4 ... 517 833   1]
 [ 11 399   1 ... 965 985   2]
 ...
 [591 932 616 ... 833 867 997]
 [249 399 973 ... 834 517 998]
 [971 931 541 ... 761 985 999]]
[[  1  11   4 ... 399 946 936]
 [  0  11   4 ... 932 936 946]
 [ 11 399   1 ...  14  24 199]
 ...
 [591 932 616 ... 876 959 573]
 [249 399 973 ... 567 435 823]
 [971 931 541 ... 823 987 690]]


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.339336,3.135496,4.635508,2.641939,3.136678,1.937022,2.338958,2.139918,3.138258,2.739921,...,3.640967,1.843438,0.437322,2.241916,2.341858,2.341535,1.438221,0.533456,1.340918,3.438753
1,3.523317,3.223526,4.321664,2.924619,3.620137,2.520902,2.023438,1.425439,3.320781,2.523918,...,3.025386,1.626592,0.620824,2.425094,2.823962,2.126022,1.323384,0.623268,1.020368,3.521498
2,3.898945,2.597887,4.697854,3.797163,2.797054,2.895825,2.997439,3.198952,2.197166,3.898414,...,3.299092,2.998163,0.197327,2.898331,2.298527,2.898016,1.896899,0.698583,0.697495,3.197118
3,3.612872,3.112963,3.912647,3.612255,2.412450,1.812786,3.012809,2.211738,1.612439,4.112579,...,1.812244,1.812601,0.511705,1.213110,1.212034,2.813278,1.112642,0.911758,0.512847,2.212257
4,3.583691,2.284768,4.480335,3.284879,2.981663,2.579810,2.782510,2.085802,2.383668,2.486832,...,2.985583,2.089049,0.178549,2.388259,2.184352,3.085719,1.483941,1.080047,0.682717,2.681369
5,3.496959,2.400169,4.697890,3.197939,3.197654,1.896424,2.498007,2.398066,2.898931,2.199014,...,3.499985,1.798729,0.298471,1.598967,2.400974,2.798191,1.298344,1.198175,0.397960,2.999495
6,3.734086,2.637492,4.834128,4.234624,3.136602,2.833323,2.733968,2.534796,2.034142,3.834762,...,2.734996,2.933377,0.232787,2.132545,2.035204,3.735613,1.435030,0.632153,0.333803,2.434158
7,3.943451,3.543054,4.443315,4.143806,3.442457,1.741205,2.743366,1.943480,1.443199,3.243198,...,2.142409,3.043975,0.342588,1.445069,1.542937,3.443590,1.441440,0.743178,0.243031,1.343242
8,3.831788,3.033350,4.832614,2.231498,2.531942,2.630922,2.331431,2.233584,3.133298,3.431947,...,2.234978,1.231768,0.432319,1.231876,2.933976,1.432484,1.333211,3.033729,0.032892,3.732310
9,4.258346,3.160653,4.459412,2.557861,3.656791,3.358707,2.958246,1.557327,3.659811,2.959735,...,0.559688,0.458288,0.058896,0.558187,2.160538,2.256732,0.857826,1.461344,0.558221,3.959869


In [227]:
basic_mean(A.T, 'COS', 2)

array([[ 0.87554259,  0.87554259,  2.3463208 ,  0.817099  ,  1.817099  ],
       [ 0.76053179,  1.28073765,  2.19991423,  3.24032594,  1.63929666],
       [ 1.05819485, -0.91557369,  3.07131058,  1.09754204,  3.08442631],
       [ 3.27377025,  3.76426215,  4.74524595,  1.25475405,  3.23573785],
       [ 1.74774972,  2.75375047, -0.24924991,  4.24924991,  2.24924991],
       [ 3.50363719,  4.00181859,  1.49636281,  1.99090703,  2.00545578],
       [ 2.15749846,  3.11022797,  1.59841035,  4.15749846,  4.59841035],
       [ 1.08516188,  3.08272382,  3.58638092,  1.58150478,  4.58150478],
       [ 4.24020355,  0.24020355,  3.75979645,  1.24020355,  2.75979645],
       [ 0.95185309,  1.44934461,  2.45937853,  3.45436157,  1.96690398]])

In [228]:
basic_baseline(A.T, 'COS', 2)

[[8 7 6 3 9 5 4 2 1 0]
 [4 7 5 6 2 9 8 3 0 1]
 [7 3 5 1 9 8 4 6 0 2]
 [8 2 7 0 9 6 5 4 1 3]
 [1 5 7 9 6 2 8 0 3 4]
 [6 4 7 2 1 9 3 8 0 5]
 [5 9 7 4 1 8 0 3 2 6]
 [2 9 4 5 6 0 3 1 8 7]
 [3 0 6 2 7 9 4 1 5 8]
 [7 4 6 2 3 5 8 1 0 9]]
[[8 7]
 [4 7]
 [7 3]
 [8 2]
 [1 5]
 [6 4]
 [5 9]
 [2 9]
 [3 0]
 [7 4]]


array([[ 0.87554259,  0.87554259,  2.3463208 ,  0.817099  ,  1.817099  ],
       [ 0.76053179,  1.28073765,  2.19991423,  3.24032594,  1.63929666],
       [ 1.05819485, -0.91557369,  3.07131058,  1.09754204,  3.08442631],
       [ 3.27377025,  3.76426215,  4.74524595,  1.25475405,  3.23573785],
       [ 1.74774972,  2.75375047, -0.24924991,  4.24924991,  2.24924991],
       [ 3.50363719,  4.00181859,  1.49636281,  1.99090703,  2.00545578],
       [ 2.15749846,  3.11022797,  1.59841035,  4.15749846,  4.59841035],
       [ 1.08516188,  3.08272382,  3.58638092,  1.58150478,  4.58150478],
       [ 4.24020355,  0.24020355,  3.75979645,  1.24020355,  2.75979645],
       [ 0.95185309,  1.44934461,  2.45937853,  3.45436157,  1.96690398]])

In [229]:
basic_mean(A.T, 'COS', 2)

array([[ 0.87554259,  0.87554259,  2.3463208 ,  0.817099  ,  1.817099  ],
       [ 0.76053179,  1.28073765,  2.19991423,  3.24032594,  1.63929666],
       [ 1.05819485, -0.91557369,  3.07131058,  1.09754204,  3.08442631],
       [ 3.27377025,  3.76426215,  4.74524595,  1.25475405,  3.23573785],
       [ 1.74774972,  2.75375047, -0.24924991,  4.24924991,  2.24924991],
       [ 3.50363719,  4.00181859,  1.49636281,  1.99090703,  2.00545578],
       [ 2.15749846,  3.11022797,  1.59841035,  4.15749846,  4.59841035],
       [ 1.08516188,  3.08272382,  3.58638092,  1.58150478,  4.58150478],
       [ 4.24020355,  0.24020355,  3.75979645,  1.24020355,  2.75979645],
       [ 0.95185309,  1.44934461,  2.45937853,  3.45436157,  1.96690398]])

In [290]:
COS(A.T)[1,4]

0.9697622757528539

In [303]:
def basic_baseline(data, sim, k):
    # initializing (1000, 1000) numpy array with zeros
    predicted_rating = np.zeros(data.shape)

    # calculating means
    mean = np.nanmean(np.where(data != 0, data, np.nan))  # the mean of all ratings
    mean_u = np.nanmean(np.where(data != 0, data, np.nan), axis=1)  # the mean of all users
    mean_i = np.nanmean(np.where(data != 0, data, np.nan), axis=0)  # the mean of all items

    # base user, item
    b_u = mean_u - mean
    b_i = mean_i - mean
    # b_ui = mean + b_u + b_i

    # selecting similarity fuction
    if sim == 'COS':
        sim = COS(data)
    elif sim == 'PCC':
        sim = PCC(data)
        
    # selecting top k users by sorting similarity array
    k_users = np.argsort(-sim)
    k_users = np.delete(k_users, np.s_[k:], 1)  # delete colomn under k

    # number of users with axis = 0 condition
    num_users = np.size(data, axis=0)
    num_items = np.size(data, axis=1)

    
    # calculating predicted ratings
    for u in range(0, num_users):
        list_sim = sim[u, k_users[u]]  # selecting top k similarity
        for i in range(0, num_items):
            list_rating = data[k_users[u], i].astype('float64')  # k users' ratings on item i

            b_ui = mean + b_u[u] + b_i[i]  # scalar
            b_vi = mean + b_u[k_users[u]] + b_i[i]  # list

            # explanation of varialbles
            # mean_u[u] : user u의 평균
            # mean_i[i] : item i의 평균
            # b_u[u] : user u의 baseline
            # b_i[i] : item i의 baseline

            # calculation
            mom = np.sum(list_sim)  # 분모
            son = np.sum(list_sim * (list_rating - b_vi))  # 분자
            predicted_rating[u, i] = b_ui + son / mom

    return predicted_rating

In [284]:
A[0,7]

0

In [304]:
basic_baseline(A, 'COS', 2)

array([[ 0.28988047,  2.81978035,  2.28988047,  1.75998059,  1.32974698,
         2.28988047,  4.30981372,  1.75998059,  2.81978035,  3.77991384],
       [ 2.74888141,  1.72480939,  2.21879139,  4.2308274 ,  1.24286341,
         3.2308274 ,  4.7368454 ,  2.20675538,  3.24286341,  2.70073737],
       [ 0.18909227,  2.65715391,  2.18909227,  1.72103063,  1.14650779,
         2.18909227,  4.16780003,  1.72103063,  2.65715391,  3.69973839],
       [-0.08579346,  2.44582768,  1.91420654,  1.3825854 ,  0.95636806,
         1.91420654,  3.9352873 ,  1.3825854 ,  2.44582768,  3.40366616],
       [ 1.95132724,  2.47897423,  1.47107509,  1.95132724,  2.47107509,
         2.46317595,  4.96712552, -0.03287448,  3.96712552,  1.47897423]])

In [305]:
basic_baseline(A, 'PCC', 2)

array([[-0.01190878,  3.28877715,  1.98809122,  0.68740529,  2.05567246,
         1.98809122,  4.52188184,  0.68740529,  3.28877715,  3.22119591],
       [ 4.28571429,  1.28571429,  1.28571429,  4.28571429,  2.28571429,
         3.28571429,  5.28571429,  0.28571429,  4.28571429,  0.28571429],
       [ 0.76388889,  1.76388889,  2.76388889,  3.76388889, -0.23611111,
         2.76388889,  3.76388889,  3.76388889,  1.76388889,  4.76388889],
       [ 0.51388889,  1.51388889,  2.51388889,  3.51388889, -0.48611111,
         2.51388889,  3.51388889,  3.51388889,  1.51388889,  4.51388889],
       [ 3.23998233,  2.23113383,  0.9036995 ,  3.23998233,  2.89927525,
         3.89927525,  3.57626516,  1.23113383,  3.23998233,  0.89927525]])

In [299]:
A

array([[4, 1, 1, 4, 2, 3, 5, 0, 4, 0],
       [0, 4, 2, 0, 3, 2, 5, 0, 4, 3],
       [2, 0, 4, 5, 2, 0, 1, 3, 4, 2],
       [1, 4, 0, 1, 4, 5, 0, 3, 1, 2],
       [1, 2, 3, 4, 0, 3, 4, 4, 2, 5]])

In [300]:
basic_baseline(A, 'COS', 2)

[[1 4 2 3 0]
 [0 4 3 2 1]
 [4 1 0 3 2]
 [1 4 2 0 3]
 [1 0 2 3 4]]
[[1 4]
 [0 4]
 [4 1]
 [1 4]
 [1 0]]
5
10
3.0607142857142855
* 2.886111111111111


array([[ 0.28988047,  2.81978035,  2.28988047,  1.75998059,  1.32974698,
         2.28988047,  4.30981372,  1.75998059,  2.81978035,  3.77991384],
       [ 2.74888141,  1.72480939,  2.21879139,  4.2308274 ,  1.24286341,
         3.2308274 ,  4.7368454 ,  2.20675538,  3.24286341,  2.70073737],
       [ 0.18909227,  2.65715391,  2.18909227,  1.72103063,  1.14650779,
         2.18909227,  4.16780003,  1.72103063,  2.65715391,  3.69973839],
       [-0.08579346,  2.44582768,  1.91420654,  1.3825854 ,  0.95636806,
         1.91420654,  3.9352873 ,  1.3825854 ,  2.44582768,  3.40366616],
       [ 1.95132724,  2.47897423,  1.47107509,  1.95132724,  2.47107509,
         2.46317595,  4.96712552, -0.03287448,  3.96712552,  1.47897423]])

In [221]:
A

array([[4, 1, 1, 4, 2, 3, 5, 0, 4, 0],
       [0, 4, 2, 0, 3, 2, 5, 0, 4, 3],
       [2, 0, 4, 5, 2, 0, 1, 3, 4, 2],
       [1, 4, 0, 1, 4, 5, 0, 3, 1, 2],
       [1, 2, 3, 4, 0, 3, 4, 4, 2, 5]])

In [113]:
basic_baseline(MovieLens_pivot, 'COS', 2)

array([[ 1.81655320e+00,  1.81655320e+00,  1.81655320e+00, ...,
        -6.80315149e-01, -6.80315149e-01,  1.31717953e+00],
       [ 3.42179744e+00,  3.92253386e+00,  3.42179744e+00, ...,
        -5.76729721e-01,  1.91958819e+00, -5.76729721e-01],
       [ 4.58647989e+00,  5.08648408e+00,  5.08648408e+00, ...,
         8.64840756e-02,  8.64840756e-02,  2.08650080e+00],
       ...,
       [ 3.98392408e+00,  4.48403460e+00,  3.48381356e+00, ...,
         1.48447670e+00, -5.15965395e-01,  1.48359251e+00],
       [ 3.93045873e+00,  4.43023094e+00,  4.43023094e+00, ...,
        -5.69769058e-01, -5.69769058e-01,  3.43023094e+00],
       [ 5.00296743e+00,  2.50221309e+00,  2.50372176e+00, ...,
         2.96742717e-03,  2.00236396e+00,  2.96742717e-03]])

In [ ]:
basic_baseline(MovieLens_pivot, 'PCC', 2)

In [214]:
-6.80315149e-01

-0.680315149

In [213]:
-5.69769058e-01

-0.569769058

In [170]:
basic_baseline(A, 'COS', 2)

array([[ 0.28988047,  2.81978035,  2.28988047,  1.75998059,  1.32974698,
         2.28988047,  4.30981372,  1.75998059,  2.81978035,  3.77991384],
       [ 2.74888141,  1.72480939,  2.21879139,  4.2308274 ,  1.24286341,
         3.2308274 ,  4.7368454 ,  2.20675538,  3.24286341,  2.70073737],
       [ 0.18909227,  2.65715391,  2.18909227,  1.72103063,  1.14650779,
         2.18909227,  4.16780003,  1.72103063,  2.65715391,  3.69973839],
       [-0.08579346,  2.44582768,  1.91420654,  1.3825854 ,  0.95636806,
         1.91420654,  3.9352873 ,  1.3825854 ,  2.44582768,  3.40366616],
       [ 1.95132724,  2.47897423,  1.47107509,  1.95132724,  2.47107509,
         2.46317595,  4.96712552, -0.03287448,  3.96712552,  1.47897423]])